In [1]:
import numpy as np 
import pandas as pd
from utils.utils import *
from utils import plotting as plotting

In [2]:
mmfit = load_mmfit()
har_data = load_crossfit()
mmfit['dataset'] = 'mmfit'
har_data['dataset'] = 'har_data'

  0%|          | 0/21 [00:00<?, ?it/s]/Users/jacobgottesman/Public/DS 4440/smartwatch-activity-recognition/utils/utils.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  raw_data = pd.concat([raw_data, df], axis =0)
  0%|          | 0/11 [00:00<?, ?it/s]/Users/jacobgottesman/Public/DS 4440/smartwatch-activity-recognition/utils/utils.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], axis = 0)
100%|██████████| 11/11 [00:04<00:00,  2.41it/s]
0it [00:00, ?it/s]
0it

In [4]:
both_df = pd.concat([mmfit, har_data], axis=0)
both_df = both_df.reset_index(drop=True)
both_df.head()

,index,frame,timestamp,acc_X,acc_Y,acc_Z,gyr_X,gyr_Y,gyr_Z,label,user_id,dataset
0,0,4050.0,1.562791e+12,9.771680,-0.507744,7.666458,-0.183259,-0.207694,-0.531453,squats,0,mmfit
1,1,4050.0,1.562791e+12,10.008787,-0.356858,8.392149,-0.197920,-0.175929,-0.477696,squats,0,mmfit
2,2,4050.0,1.562791e+12,10.097403,-0.344883,9.022039,-0.111177,-0.146608,-0.422719,squats,0,mmfit
3,3,4051.0,1.562791e+12,10.229129,-0.011975,9.005274,0.008552,-0.108734,-0.362854,squats,0,mmfit
4,5,4051.0,1.562791e+12,11.127261,0.560435,8.715476,0.162490,-0.051313,-0.155160,squats,0,mmfit


In [6]:
print(f'There are {len(both_df['user_id'].unique())} unique users.')
print(f'There are {len(both_df['label'].unique())} unique labels.')

There are 78 unique user.
There are 20 unique labels.


In [7]:
both_df.isna().sum()

index        1538606
frame        1538606
timestamp    1538606
acc_X              0
acc_Y              0
acc_Z              0
gyr_X              0
gyr_Y              0
gyr_Z              0
label              0
user_id            0
dataset            0
dtype: int64

In [11]:
both_df['label'].value_counts()

label
non-e                      1149916
burpees                     189031
pushups                     188069
squats                      180007
boxjumps                    178455
deadlifts                   173752
crunches                    161433
wallballs                   143426
kbsquatpress                127833
kbpress                     121313
pullups                      84666
null                         67936
situps                       58318
lunges                       57296
dumbbell_shoulder_press      48316
tricep_extensions            41331
lateral_shoulder_raises      39737
dumbbell_rows                36487
bicep_curls                  35654
jumping_jacks                19411
Name: count, dtype: int64

In [10]:
# create completely seperate 2 second pieces of data for testing/training
model_data = create_examples(both_df, seconds=2)

100%|██████████| 3102387/3102387 [03:48<00:00, 13576.09it/s]


In [12]:
print(f'There are {len(model_data)} rows of data for training/testing')
print('Distribution of classes within data for model')
model_data['label'].value_counts()

There are 14667 rows of data for training/testing
Distribution of classes within data for model


label
non-e                      5456
burpees                     919
pushups                     874
boxjumps                    864
squats                      846
deadlifts                   836
crunches                    783
wallballs                   693
kbsquatpress                615
kbpress                     581
pullups                     400
null                        335
situps                      259
lunges                      257
dumbbell_shoulder_press     211
tricep_extensions           179
lateral_shoulder_raises     172
dumbbell_rows               156
bicep_curls                 154
jumping_jacks                77
Name: count, dtype: int64

In [13]:
model_data.to_csv('data/2_sec_model_data.csv')
both_df.to_csv('data/mmfit_crossfit_data.csv')

In [14]:
both_df.shape

(3102387, 12)